In [18]:
from multiprocessing import Process, Pipe
import signal
import logging
import random
import numpy as np

In [19]:
filename = 'ch_log.txt'
with open(filename, 'w+') as f:
    f.write('')
f.close()
msg_filename = 'ch_msg.txt'
with open(msg_filename, 'w+') as f:
    f.write('')
f.close()

## Logging

* `logger.setLevel(logging.DEBUG)` to debug
* `logger.setLevel(logging.WARNING)` to only log the final result (deadlocked or not deadlocked)

In [20]:
logging.basicConfig()
logger = logging.getLogger('MPLogger')
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.WARNING)

## Set parameters

In [22]:
num_processes = 5
wfg = deadlocked_wfg
initiator = 2
timeout = 1
msg_gaurentee = 0.99
num_experiments = 100 # Number of times to simulate

## Timeout

* In diffusion based methods, if initiator receives messages for all processes in its dependent set, it is deadlocked.
* Else if it did not receive all messages before timeout, it will assume that it is not deadlocked
* Noisy channel with less than 100% delivery gaurentee can only mistake deadlocked situation as not deadlocked, vice versa cannot happen

In [23]:
class Timeout:
    def __init__(self, seconds=1, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message
    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)
    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)
    def __exit__(self, type, value, traceback):
        signal.alarm(0)

## Message passing gaurentees

* if prob == 1, then gaurentee is 100%

In [24]:
def noisy_send(pipe, msg):
    send_prob = np.random.uniform(0, 1)
    if send_prob <= msg_gaurentee:
        pipe.send(msg)
        logger.debug("Succesfully sent: {}".format(msg) + " Prob generated: {}".format(send_prob))
        with open(msg_filename, 'a+') as f: f.write('1')
    else:
        logger.debug("Failed: {}".format(msg) + " Prob generated: {}".format(send_prob))

## Dependency lists

* Funcion to get dependency lists for each process

In [25]:
def get_waiting_on(id, wfg):
    waiting_on = []
    for pi, pj in wfg:
        if pj == id:
            waiting_on.append(pi)
    return waiting_on

In [26]:
def get_waited_by(id, wfg):
    waited_by = []
    for pi, pj in wfg:
        if pi == id:
            waited_by.append(pj)
    return waited_by

## Chandy-Herman

In [27]:
def is_subset(T, B):
    for i, _ in enumerate(T):
        if not T[i] == B[i]:
            return False
    return True

In [28]:
def find_subset(iq_list, B):
    for i in range(len(iq_list)):
        if is_subset(iq_list[i][1], B):
            return i
    return None

In [29]:
def find_match(oq_list, B):
    for i in range(len(oq_list)):
        if oq_list[i][1] == B:
            return i
    return None

In [30]:
def initiator_process(id, send_pipes, recv_pipes, query_id, timeout=10):
    
    try:
        with Timeout(timeout):
            noisy_send(send_pipes[query_id], ("query", [id], id))

            while True:
                if recv_pipes[query_id].poll():
                    logger.error("Deadlocked!")
                    with open(filename, 'a+') as f: f.write("{} {}\n".format(msg_gaurentee, 1))
                    return
    except TimeoutError as e:
        logger.error("Not deadlocked!")
        with open(filename, 'a+') as f: f.write("{} {}\n".format(msg_gaurentee, 0))

In [31]:
def process(id, num_processes, send_pipes, recv_pipes, waiting_on, timeout=10):
    np.random.seed(random.randint(0, 100000))
        
    logger.debug("{} waiting_on {}".format(id, waiting_on))
    
    iq_list = []
    oq_list = []
    
    try:
        with Timeout(timeout):

            while True:
                for recv_id, conn in enumerate(recv_pipes):
                    while conn.poll():
                        type_, B, w = conn.recv()
                        
                        # ====================================================================
                        
                        if len(waiting_on) > 0: # if True, the process is blocked
                            if type_ == 'query':
                                match = find_subset(iq_list, B)
                                if match is not None: # reflection
                                    noisy_send(send_pipes[w], ("reply", B, id))
                                else: # extension
                                    iq_list.append((type_, B, w))
                                    for pj in waiting_on:
                                        noisy_send(send_pipes[pj], ("query", B, id))
                                        oq_list.append(("query", B, id))
                                    
                        # ====================================================================

                        if type_ == 'reply':
                            match = find_match(oq_list, B)
                            if match is not None:
                                del oq_list[match]
                                match = find_match(oq_list, B)
                                if match is None:
                                    match = find_match(iq_list, B)
                                    noisy_send(send_pipes[iq_list[match][-1]], ("reply", B, id))
                                    del iq_list[match]


                        # ====================================================================
    
    except TimeoutError as e:
        pass